## Imports

In [2]:
import pandas as pd
import pickle

## Data

In [3]:
ekg_denoised = pd.read_pickle('ekg_denoised.pkl')

In [5]:
ekg_denoised.head()

,ICD9_CODE,TEXT
0,4019,sinus rhythm with 2:1 a-v block. right bundle-...
1,4019,sinus tachycardia consider old septal myocardi...
2,4019,sinus rhythm. low limb lead voltage. compared ...
3,4019,sinus rhythm. early precordial qrs transition ...
5,4019,sinus bradycardia. non-specific inferior t wav...
